# Assignment
Pick a dataset of your choice with a binary outcome and the potential for at least 15 features. If you're drawing a blank, the crime rates in 2013 dataset has a lot of variables that could be made into a modelable binary outcome.

Engineer your features, then create three models. Each model will be run on a training set and a test-set (or multiple test-sets, if you take a folds approach). The models should be:

Vanilla logistic regression,
Ridge logistic regression,
Lasso logistic regression.

In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?

# Report
I chose the 2013 NY FBI crime rate database, because it had a modelable binary outcome in the "Murder_and_nonnegligent_manslaughter" column, which is "0" for no murders in a given NY jurisdiction in 2013 and "1" for 1 or more murders in the jurisdiction.

I chose the 2013 NY FBI crime rate database as a training set and the 2014 NY FBI crime rate database as a test set.

I chose features which I had previously used to model property crimes, on the hunch that property crimes and non-property crimes (including murder) might be correlated with the same features.

I also intentionally chose some features that might be highly correlated with each other, such as 
Violent Crime and Aggravated Assault, because I thought the Ridge Regression method in particular would be useful in getting rid of coefficient inflation caused by overly-correlated features.

The features I chose were:
'Population','Robbery','Property_crime','Motor_vehicle_theft','Violent_crime','Aggravated_assault', and 'Burglary.'
The modelable binary outcome variable was 'Murder_and_nonnegligent_manslaughter.'

I ran these features using "Vanilla" Binary Logistic regression, Ridge logistic regression, and Lasso Logistic Regression.

For Binary Logistic regression, I did not modify the default "penalty" parameter in SKLearn's Logistic Regression method. The penalty parameter appears to be either 'l1' for Lasso Regression or 'l2' for Ridge Regresssion. There is no indication in the documentation of an available 'none' penalty for Binary Logistic Regression, so I used a large number for the "C" parameter (inverse of regularization strength) in the case of Binary Logistic Regression.
For Ridge Regression, I specified the 'l2' penalty parameter.
For LASSO Regression, I specified the 'l1' penalty parameter.

Surprisingly, none of the features appeared to be worth dropping - neither Ridge nor LASSO regression improved on the performance of Binary Logistic regression in this case, and Lasso Regression did not force any features to be nearly zero. 

I decided to try the three techniques for modelling a non-binary outcome - in other words, using the three techniques as regressors rather than classifiers. The models retained comparable performance as regressors, both as compared to their performance as classifiers, and compared to each other, using r-squared as a model evaluation criterion.


In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
pd.options.display.float_format = '{:.3f}'.format
# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

/usr/local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Parse in the dataframes, skipping footnotes
df2013_raw = pd.read_csv('NYState2013orig.csv',skiprows=4,skipfooter=3)
df2014_raw = pd.read_csv('NYState2014.csv',skiprows=4,skipfooter=7)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
df2013_raw.tail()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
343,Woodbury Town,"10,685",3,0,nan,0,2,1,541,9,529,3,nan
344,Woodridge Village,829,7,0,nan,0,0,7,17,8,9,0,0.000
345,Woodstock Town,"5,931",2,0,nan,0,0,2,58,13,45,0,nan
346,Yonkers,"199,134","1,036",6,nan,25,390,615,"2,368",470,"1,662",236,10.000
347,Yorktown Town,"36,643",15,0,nan,0,2,13,334,45,287,2,nan


In [4]:
df2014_raw.tail()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
364,Woodbury Town4,"10,739",4,0,0,nan,1,3,NaN,5,NaN,0,0.000
365,Woodstock Town4,"5,907",3,0,1,nan,2,0,43,14,29,0,0.000
366,Yonkers4,"200,624",974,3,33,nan,358,580,"2,009",414,"1,395",200,15.000
367,Yorktown Town4,"36,989",13,0,0,nan,0,13,209,24,182,3,0.000
368,Youngstown Village4,"1,896",0,0,0,nan,0,0,1,0,1,0,0.000


In [5]:
def convert_value(num):
    if num > 0:
        return 1
    else: 
        return 0

In [6]:
def process_crime_stats(df):
    # Convert Population column to strings.
    df.Population=df.Population.apply(lambda x:str(x))
    # Take out commas in Population column
    df.Population = df.Population.str.replace(',','')
    # Make Population an integer
    df.Population=df.Population.apply(lambda x:int(x))
    # Create a Population squared column
    df['Population_sq']=df.Population**2
    # Fix column headings
    df.columns = [c.replace(' ', '_') for c in df.columns]
    df.columns = [c.replace('\n', '_') for c in df.columns]
    # Fix Property Crime column
    # drop rows where Property_crime is "na"
    df=df.loc[~df.Property_crime.isna()].copy()
    df.Property_crime = df.Property_crime.apply(lambda x:str(x))
    df.Property_crime = df.Property_crime.str.replace(',','')
    df.Property_crime = df.Property_crime.apply(lambda x:int(x))
    # Fix Robbery column
    df.Robbery=df.Robbery.apply(lambda x:str(x))
    df.Robbery = df.Robbery.str.replace(',','')
    df.Robbery=df.Robbery.apply(lambda x:int(x))
    # Before making Robbery categorical, make a copy of the column for later:
    df['RobberyNum']=df['Robbery']
    # Make Robbery categorical
    df.Robbery=df.Robbery.apply(lambda x:convert_value(x))
    # Before making murder categorical, make a copy of the column for later:
    df['MurderNum']=df['Murder_and_nonnegligent_manslaughter']
    # Make the murder column categorical
    df.Murder_and_nonnegligent_manslaughter=df.Murder_and_nonnegligent_manslaughter.apply(lambda x:convert_value(x))
    # Fix Violent_crime column
    df=df.loc[~df.Violent_crime.isna()].copy()
    df.Violent_crime = df.Violent_crime.apply(lambda x:str(x))
    df.Violent_crime = df.Violent_crime.str.replace(',','')
    df.Violent_crime = df.Violent_crime.apply(lambda x:int(x))
    # Fix Burglary column
    df=df.loc[~df.Burglary.isna()].copy()
    df.Burglary = df.Burglary.apply(lambda x:str(x))
    df.Burglary = df.Burglary.str.replace(',','')
    df.Burglary = df.Burglary.apply(lambda x:int(x))
    # Fix Motor_vehicle_theft column
    df=df.loc[~df.Motor_vehicle_theft.isna()].copy()
    df.Motor_vehicle_theft = df.Motor_vehicle_theft.apply(lambda x:str(x))
    df.Motor_vehicle_theft = df.Motor_vehicle_theft.str.replace(',','')
    df.Motor_vehicle_theft = df.Motor_vehicle_theft.apply(lambda x:int(x))
    # Fix Aggravated_assault column
    df=df.loc[~df.Aggravated_assault.isna()].copy()
    df.Aggravated_assault = df.Aggravated_assault.apply(lambda x:str(x))
    df.Aggravated_assault = df.Aggravated_assault.str.replace(',','')
    df.Aggravated_assault = df.Aggravated_assault.apply(lambda x:int(x))
    return df

In [7]:
df2013_processed = process_crime_stats(df2013_raw)

In [8]:
df2014_processed = process_crime_stats(df2014_raw)

In [9]:
df2013_processed.head()

,City,Population,Violent_crime,Murder_and_nonnegligent_manslaughter,Rape_(revised_definition)1,Rape_(legacy_definition)2,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny-_theft,Motor_vehicle_theft,Arson3,Population_sq,RobberyNum,MurderNum
0,Adams Village,1861,0,0,nan,0,0,0,12,2,10,0,0.000,3463321,0,0
1,Addison Town and Village,2577,3,0,nan,0,0,3,24,3,20,1,0.000,6640929,0,0
2,Akron Village,2846,3,0,nan,0,0,3,16,1,15,0,0.000,8099716,0,0
3,Albany,97956,791,1,nan,30,1,526,4090,705,"3,243",142,nan,9595377936,227,8
4,Albion Village,6388,23,0,nan,3,1,16,223,53,165,5,nan,40806544,4,0


In [10]:
df2013_features=df2013_processed[['Population','Robbery','Property_crime','Motor_vehicle_theft','Violent_crime',
                                  'Aggravated_assault','Burglary']]

In [11]:
df2013_target=df2013_processed['Murder_and_nonnegligent_manslaughter']

In [12]:
df2014_features=df2014_processed[['Population','Robbery','Property_crime','Motor_vehicle_theft','Violent_crime',
                                  'Aggravated_assault','Burglary']]

In [13]:
df2014_target=df2014_processed['Murder_and_nonnegligent_manslaughter']

In [14]:
# "Vanilla" Logistic Regression - note "C" is inverse of regularization strength, so high C value here denotes
# absence of a regularization penalty.
# Declare a logistic regression classifier with
# parameter regularization coefficient C.
regrV = LogisticRegression(C=1e9)

print('Cross-validating a Binary Logistic Regression model between two different data sets: \n')

# Fit our model to our training data.
regrV.fit(df2013_features,df2013_target)

print('Coefficients: \n', regrV.coef_)
print('Intercept: \n', regrV.intercept_)

# Performance on the training data:
print('R squared of model on 2013 training data: \n',regrV.score(df2013_features,df2013_target)) 

# Performance on the test data:
# Score is measuring predicted outcome (same as regr.predict()) v. test data.
print('R squared of model on 2014 test data: \n',regrV.score(df2014_features,df2014_target))

Cross-validating a Binary Logistic Regression model between two different data sets: 

Coefficients: 
 [[  3.86382954e-05  -5.06802396e-01  -7.57246598e-04  -4.85825949e-02
    4.56814640e-02  -3.90112025e-02   7.95088934e-03]]
Intercept: 
 [-2.60194314]
R squared of model on 2013 training data: 
 0.905172413793
R squared of model on 2014 test data: 
 0.910326086957


In [15]:
# Ridge Logistic Regression - default penalty in SKLearn is L2 regularization, which is Ridge Regression
regrR = LogisticRegression(penalty='l2')
print('Cross-validating a Ridge Logistic Regression model between two different data sets: \n')

# Fit our model to our training data.
regrR.fit(df2013_features,df2013_target)

print('Coefficients: \n', regrR.coef_)
print('Intercept: \n', regrR.intercept_)

# Performance on the training data:
print('R squared of model on 2013 training data: \n',regrR.score(df2013_features,df2013_target)) 

# Performance on the test data:
# Score is measuring predicted outcome (same as regr.predict()) v. test data.
print('R squared of model on 2014 test data: \n',regrR.score(df2014_features,df2014_target))

Cross-validating a Ridge Logistic Regression model between two different data sets: 

Coefficients: 
 [[  3.54309159e-05  -4.93383602e-01  -6.66853409e-04  -4.71805908e-02
    4.68262982e-02  -4.06769075e-02   6.99936760e-03]]
Intercept: 
 [-2.49527237]
R squared of model on 2013 training data: 
 0.902298850575
R squared of model on 2014 test data: 
 0.910326086957


In [16]:
# LASSO Logistic Regression - specifying L1 penalty in SKLearn, which is LASSO Regression
regrL = LogisticRegression(penalty='l1')
print('Cross-validating a LASSO Logistic Regression model between two different data sets: \n')

# Fit our model to our training data.
regrL.fit(df2013_features,df2013_target)

print('Coefficients: \n', regrL.coef_)
print('Intercept: \n', regrL.intercept_)

# Performance on the training data:
print('R squared of model on 2013 training data: \n',regrL.score(df2013_features,df2013_target)) 

# Performance on the test data:
# Score is measuring predicted outcome (same as regr.predict()) v. test data.
print('R squared of model on 2014 test data: \n',regrL.score(df2014_features,df2014_target))

Cross-validating a LASSO Logistic Regression model between two different data sets: 

Coefficients: 
 [[  2.18122556e-05   6.25807659e-01  -1.69208576e-04  -3.15863739e-03
    6.43179774e-03   6.21029045e-03   2.55727601e-03]]
Intercept: 
 [-3.1493986]
R squared of model on 2013 training data: 
 0.899425287356
R squared of model on 2014 test data: 
 0.913043478261


In [17]:
# Let's try LASSO Logistic Regression on MurderNum, which is the number of murders, rather than the binary outcome
# represented by Murder_and_nonnegligent_manslaughter
df2013_features=df2013_processed[['Population','Robbery','Property_crime','Motor_vehicle_theft','Violent_crime',
                                  'Aggravated_assault','Burglary']]
df2013_target=df2013_processed['MurderNum']
df2014_features=df2014_processed[['Population','Robbery','Property_crime','Motor_vehicle_theft','Violent_crime',
                                  'Aggravated_assault','Burglary']]
df2014_target=df2014_processed['MurderNum']
# LASSO Logistic Regression - specifying L1 penalty in SKLearn, which is LASSO Regression
regrL = LogisticRegression(penalty='l1')
print('Cross-validating a LASSO Logistic Regression model between two different data sets: \n')

# Fit our model to our training data.
regrL.fit(df2013_features,df2013_target)

print('Coefficients: \n', regrL.coef_)
print('Intercept: \n', regrL.intercept_)

# Performance on the training data:
print('R squared of model on 2013 training data: \n',regrL.score(df2013_features,df2013_target)) 

# Performance on the test data:
# Score is measuring predicted outcome (same as regr.predict()) v. test data.
print('R squared of model on 2014 test data: \n',regrL.score(df2014_features,df2014_target))

Cross-validating a LASSO Logistic Regression model between two different data sets: 

Coefficients: 
 [[ -3.60299197e-05  -1.37758236e+00   9.46376181e-04   3.80230617e-02
   -3.16699522e-02   1.94030320e-02  -6.15160120e-03]
 [  2.83809270e-05   1.49223839e+00   2.26277797e-04  -3.38745055e-02
    4.09047852e-03  -1.32566247e-02   6.60949762e-03]
 [  4.79891405e-05   0.00000000e+00  -1.88681946e-03   2.07009987e-02
   -5.23287093e-04  -9.77706021e-03   2.81149839e-03]
 [ -9.65096299e-04   0.00000000e+00   1.04637677e-02  -1.17892203e-01
    9.89760450e-03   0.00000000e+00   5.71353897e-02]
 [ -1.99744865e-04   0.00000000e+00  -2.51548749e-02   2.13686541e-01
    1.51444765e-01  -5.17541448e-01   1.50422333e-01]
 [ -3.17114194e-03   0.00000000e+00  -3.47850479e-03   1.47556101e-02
    1.33375225e-01   1.39981001e-01   1.26047170e-02]
 [  1.75975014e-06   0.00000000e+00  -6.77859678e-02   0.00000000e+00
    1.18894195e-01   9.42264677e-02  -2.88748780e-02]
 [ -5.20149230e-05   0.0000000

In [18]:
# Trying Ridge Regression on MurderNum:

df2013_features=df2013_processed[['Population','Robbery','Property_crime','Motor_vehicle_theft','Violent_crime',
                                  'Aggravated_assault','Burglary']]
df2013_target=df2013_processed['MurderNum']
df2014_features=df2014_processed[['Population','Robbery','Property_crime','Motor_vehicle_theft','Violent_crime',
                                  'Aggravated_assault','Burglary']]
df2014_target=df2014_processed['MurderNum']
# Ridge Logistic Regression - default penalty in SKLearn is L2 regularization, which is Ridge Regression
regrR = LogisticRegression(penalty='l2')
print('Cross-validating a Ridge Logistic Regression model between two different data sets: \n')

# Fit our model to our training data.
regrR.fit(df2013_features,df2013_target)

print('Coefficients: \n', regrR.coef_)
print('Intercept: \n', regrR.intercept_)

# Performance on the training data:
print('R squared of model on 2013 training data: \n',regrR.score(df2013_features,df2013_target)) 

# Performance on the test data:
# Score is measuring predicted outcome (same as regr.predict()) v. test data.
print('R squared of model on 2014 test data: \n',regrR.score(df2014_features,df2014_target))

Cross-validating a Ridge Logistic Regression model between two different data sets: 

Coefficients: 
 [[ -2.81387792e-05   5.49084557e-01   5.16063855e-04   3.79112097e-02
   -3.51006646e-02   2.26358652e-02  -5.31196246e-03]
 [  2.96430524e-05   2.10680459e-01   4.20638144e-04  -3.82064632e-02
    5.05495541e-03  -1.52415284e-02   7.00122969e-03]
 [ -5.31332134e-05  -2.54237683e-01   8.16213757e-05   1.11589279e-01
    9.02074316e-03   3.23763037e-03  -5.87241221e-02]
 [ -2.11610092e-03  -6.15809909e-01   2.25440081e-02  -3.93920489e-01
    2.91359524e-01  -3.51102602e-01   8.25292925e-02]
 [ -4.55325512e-04  -8.31647220e-01  -2.12084323e-02   2.64812630e-01
    2.82611298e-01  -8.72898937e-01   1.75839840e-01]
 [ -9.03666673e-03  -2.13922740e-03   1.62020857e-02   5.84285212e-02
    3.90128187e-01   1.80458937e-01   8.89679441e-02]
 [ -3.46542614e-05  -7.62999083e-02  -1.47805976e-01  -3.30276319e-01
    5.70886363e-01  -7.80135312e-02  -2.24016449e-01]
 [ -3.62140560e-04  -7.5518618

In [19]:
# Binary Logistic Regression on MurderNum:

df2013_features=df2013_processed[['Population','Robbery','Property_crime','Motor_vehicle_theft','Violent_crime',
                                  'Aggravated_assault','Burglary']]
df2013_target=df2013_processed['MurderNum']
df2014_features=df2014_processed[['Population','Robbery','Property_crime','Motor_vehicle_theft','Violent_crime',
                                  'Aggravated_assault','Burglary']]
df2014_target=df2014_processed['MurderNum']
regrV = LogisticRegression(C=1e9)

print('Cross-validating a Binary Logistic Regression model between two different data sets: \n')

# Fit our model to our training data.
regrV.fit(df2013_features,df2013_target)

print('Coefficients: \n', regrV.coef_)
print('Intercept: \n', regrV.intercept_)

# Performance on the training data:
print('R squared of model on 2013 training data: \n',regrV.score(df2013_features,df2013_target)) 

# Performance on the test data:
# Score is measuring predicted outcome (same as regr.predict()) v. test data.
print('R squared of model on 2014 test data: \n',regrV.score(df2014_features,df2014_target))

Cross-validating a Binary Logistic Regression model between two different data sets: 

Coefficients: 
 [[ -4.19747938e-05   3.82186025e-01   7.90325567e-04   4.47543536e-02
   -4.07035346e-02   3.22238943e-02  -7.81785652e-03]
 [  9.70411710e-06  -4.26466547e-01   1.09528023e-03  -4.07860814e-02
    1.31050338e-02  -2.35325022e-02   3.31525103e-03]
 [ -1.48185635e-04  -5.44575018e-02  -4.12511304e-03   4.47507562e-02
    3.74632620e-02   1.25421443e-02  -5.23761930e-02]
 [ -2.24845469e-03  -5.33601191e-01   2.34465252e-02  -4.27496482e-01
    3.17458979e-01  -3.81699313e-01   8.87623432e-02]
 [ -7.06916709e-05  -2.15439388e+00  -7.02100390e-02   3.07394950e-01
    6.16253351e-01  -1.57535880e+00   3.34697375e-01]
 [ -1.23779582e-02  -2.79948232e-03   2.79394718e-02   9.04782491e-02
    5.44103593e-01   2.27769613e-01   9.95802020e-02]
 [ -2.82223046e-05  -7.33646774e-02  -1.49679384e-01  -3.09223754e-01
    5.78670453e-01  -7.61975745e-02  -2.47634794e-01]
 [ -7.51796731e-04  -3.034982